In [173]:
import spacy
from spacy import displacy
#source .venv/bin/activate

In [237]:
zlp = spacy.load('zh_core_web_lg')
elp = spacy.load('en_core_web_lg')


sentence1 = "只要胡同存在一天，它便是个有机体，有生命、有感情，他会思念远人，远人也会思念它"#"众议院监督与政府改革委员会主席、来自肯塔基州的共和党人詹姆斯 于今年1月启动了相关调查"# 
sentence2 = "The day a vacuum exists, it's an organic body, it's alive, it's emotional, and it's going to think about people, and people will think about it."
#"When I came home I was tired"
# Parse the sentence using spaCy
doc1 = zlp(sentence1)
doc2 = elp(sentence2)

# Visualize the dependency tree with default settings
'''displacy.render(doc1, style='dep', jupyter=True)
displacy.render(doc2, style='dep', jupyter=True)'''

print("doc1:")
for token in doc1:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

print("doc2:")
for token in doc2:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

doc1:
只要 advmod 存在 VERB []
胡同 nsubj 存在 VERB []
存在 dep 有 VERB [只要, 胡同, 一]
一 nmod:range 存在 VERB [天]
天 mark:clf 一 NUM []
， punct 有 VERB []
它 nsubj 有 VERB []
便 advmod 有 VERB []
是 cop 有 VERB []
个 dep 有 VERB []
有 ROOT 有 VERB [存在, ，, 它, 便, 是, 个, 机体, ，, 有, ，, 会, 会]
机体 dobj 有 VERB []
， punct 有 VERB []
有 conj 有 VERB [生命, 、, 有]
生命 dobj 有 VERB []
、 punct 有 VERB []
有 conj 有 VERB [感情]
感情 dobj 有 VERB []
， punct 有 VERB []
他 nsubj 会 VERB []
会 dep 有 VERB [他, 人, ，]
思念 compound:nn 人 NOUN []
远 advmod 人 NOUN []
人 dobj 会 VERB [思念, 远]
， punct 会 VERB []
远人 advmod 会 VERB []
也 advmod 会 VERB []
会 dep 有 VERB [远人, 也, 思念]
思念 dobj 会 VERB []
它 ROOT 它 PRON []
doc2:
The det day NOUN []
day npadvmod 's AUX [The, exists]
a det vacuum NOUN []
vacuum nsubj exists VERB [a]
exists relcl day NOUN [vacuum]
, punct 's AUX []
it nsubj 's AUX []
's ccomp 's AUX [day, ,, it, body]
an det body NOUN []
organic amod body NOUN []
body attr 's AUX [an, organic]
, punct 's AUX []
it nsubj 's AUX []
's ccomp 's AUX ['s, ,, it, alive]
aliv

In [236]:
import spacy
import nltk
from nltk.grammar import CFG
from nltk.parse import RecursiveDescentParser

def get_root(doc):
    curr = doc[0]
    while curr.head != curr:
        curr = curr.head
    return curr


def to_nltk_tree(node):
    mask = "1" if node.dep_ == "nsubj" else "0"
    if len(list(node.children)) > 0:
        return nltk.Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

nltk_tree1 = to_nltk_tree(get_root(doc1))
nltk_tree2 = to_nltk_tree(get_root(doc2))
nltk_tree1.pretty_print()
nltk_tree2.pretty_print()

     疲倦            
  ___|_______       
 |   |       回家    
 |   |    ___|___   
 他   很   他       后 

            was          
  ___________|____        
 |    |          got     
 |    |       ____|___    
 he sleepy After  he home



In [200]:
def jaccard(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
     
    return intersection / union if union else 0

In [87]:
from itertools import permutations

arr = [[1, 2, 3], [4, 5, 6], [9, 1, 3]]

a = [1, 2, 2]
res = list(permutations(a))
ls = list(res[0])
[arr[i][l] for i in range(len(arr)) for l in ls]



[2, 3, 3, 5, 6, 6, 1, 3, 3]

In [214]:
import zss

class Node:
    def __init__(self, label, children=None):
        self.label = label
        self.children = children if children else []

    def get_children(node):
        return node.children

    def get_label(node):
        return node.label
    
    def complex_distance(token1, token2):
        return 0 if set(token1.children) == set(token2.children) else (len(token1.children))

def doc_to_tree(root):
    children = list(root.children)
    if len(children) == 0:
        return Node(root.dep_, [])
    else:
        return Node(root.dep_, [doc_to_tree(child) for child in children])

def pronoun_mask(tree):
    if tree.label != 'nsubj':
        tree.label = 0
    else:
        tree.label = 1
    children = tree.get_children()
    if len(children) > 0:
        for child in children: child = pronoun_mask(child)
    return tree

'''def mask(tree):
    tree.label = ['::']
    children = tree.get_children()
    if len(children) > 0:
        for child in children: mask(child)
    return tree

def collapse_pronouns(root):
    children = root.get_children()
    leaf_sum = 0
    branches = []
    for child in children:
        if not child.get_children():
            leaf_sum += child.get_label()
        else:
            branches.append(collapse_pronouns(child))
    return Node(root.get_label(), [Node(leaf_sum)]+branches) if root.get_children() else Node(root.get_label())

def unordered_distance(trees):
    dist = 0

    if len([[child.get_label() for child in tree.get_children()] for tree in trees]) == 0:
        if set([tree.get_label() for tree in trees]).size() == 0:
            return 0
        else: return 1

    for i in range(1, len(trees)): 
        children1 = trees[i-1].get_children()
        children2 = trees[i].get_children()
        labels1 = [child.get_label() for child in children1]
        labels2 = [child.get_label() for child in children2]

        if len(children1) == len(children2) and set(labels1) == set(labels2):
            print(labels1, labels2)
            n = len(children1)

            arr = [[0 for i in range(n)] for j in range(n)]
            for i in range(n):
                for j in range(n):
                    arr[i][j] = unordered_distance([children1[i], children2[j]])

            print(arr)
            perms = list(permutations([i for i in range(n)]))

            d = float("inf")
            for perm in perms:
                ps = list(perm)
                total = sum([arr[i][ps[i]] for i in range(n)])
                if total < d:
                    d = total

            dist += d

        else:
            dist+=(sum(labels1)-sum(labels2))

    return dist'''

'''tree1 = Node(1, [Node(0, [Node(1), Node(1)]), Node(0)])
tree2 = Node(1, [Node(0, [Node(1)]), Node(1)])

distance = zss.simple_distance(tree1, tree2, Node.get_children, Node.get_label)
print(f"Tree edit distance: {distance}")'''

'tree1 = Node(1, [Node(0, [Node(1), Node(1)]), Node(0)])\ntree2 = Node(1, [Node(0, [Node(1)]), Node(1)])\n\ndistance = zss.simple_distance(tree1, tree2, Node.get_children, Node.get_label)\nprint(f"Tree edit distance: {distance}")'

In [215]:
def tree_to_set(root):

    def helper(root, level):
        level = level
        elems = []
        children = root.get_children()
        labels = [child.get_label() for child in children]
        for child in children:
            elems.append((level, root.get_label(), child.get_label()))
            elems += helper(child, level + 1)
        return elems
    
    return [(0, -1, root.get_label())] + helper(root, 1)

#tree1 = Node(0, [Node(1), Node(0), Node(0, [Node(0), Node(1), Node(0)])])

In [216]:
def collapse_pronouns(root):
    children = root.get_children()
    leaf_sum = 0
    branches = []
    for child in children:
        if not child.get_children():
            leaf_sum += child.get_label()
        else:
            branches.append(collapse_pronouns(child))
    return Node(root.get_label(), [Node(leaf_sum)]+branches) if root.get_children() else Node(root.get_label())

tree1 = Node(0, [Node(1), Node(0), Node(0, [Node(0), Node(1), Node(0)])])
tree1_col = collapse_pronouns(tree1)
tree_to_set(tree1_col)

[(0, -1, 0), (1, 0, 1), (1, 0, 0), (2, 0, 1)]

In [217]:
tr1 = doc_to_tree(get_root(doc1))
#tr2 = collapse_pronouns(pronoun_mask(doc_to_tree(get_root(doc2))))
tree_to_set(collapse_pronouns(pronoun_mask(tr1)))

[(0, -1, 0),
 (1, 0, 0),
 (1, 0, 0),
 (2, 0, 1),
 (1, 0, 1),
 (2, 1, 0),
 (2, 1, 0),
 (3, 0, 0),
 (1, 0, 0),
 (2, 0, 0),
 (1, 0, 0),
 (2, 0, 0),
 (2, 0, 0),
 (3, 0, 0),
 (3, 0, 0),
 (4, 0, 0),
 (4, 0, 0),
 (5, 0, 0),
 (3, 0, 0),
 (4, 0, 0),
 (4, 0, 0),
 (5, 0, 0),
 (5, 0, 0),
 (6, 0, 0)]

In [222]:
def compare(doc1, doc2):
    set1 = set()
    set2 = set()
    ls1 = tree_to_set(collapse_pronouns(pronoun_mask(doc_to_tree(get_root(doc1)))))
    ls2 = tree_to_set(collapse_pronouns(pronoun_mask(doc_to_tree(get_root(doc2)))))
    print(ls1)
    for el in ls1:
        set1.add(el)
    for el in ls2:
        set2.add(el)
    return jaccard(set1, set2)

In [224]:
compare(doc1, doc2)

[(0, -1, 0), (1, 0, 1), (1, 0, 0), (2, 0, 1), (2, 0, 0), (3, 0, 0), (1, 0, 0), (2, 0, 0), (2, 0, 0), (3, 0, 0), (1, 0, 0), (2, 0, 1), (2, 0, 0), (3, 0, 0), (1, 0, 0), (2, 0, 0)]


0.5454545454545454

In [63]:
set1 = set()
set2 = set()
set1.add('a')
set1.add('b')
set2.add('b')
set2.add('a')
print(set1, set2)
print(set1 == set2)
print(Node('g') == Node('g'))

{'b', 'a'} {'b', 'a'}
True
False


In [ ]:
doc3 = zlp("只有把问题想得严重一些,  才能跑赢疫情传播速度")
doc4 = elp("The only way to beat the spread of the epidemic is to think more seriously.")

compare(doc3, doc4)

to_nltk_tree2(get_root(doc3)).pretty_print()
to_nltk_tree2(get_root(doc4)).pretty_print()


[(0, -1, 0), (1, 0, 0), (1, 0, 0), (2, 0, 0), (1, 0, 0), (2, 0, 0), (2, 0, 0), (3, 0, 0)]
                 严重                        
  _______________|_______________           
 |   |   |   |   |   |   |       跑赢        
 |   |   |   |   |   |   |    ___|___       
 |   |   |   |   |   |   ,   |       速度    
 |   |   |   |   |   |   |   |    ___|___   
 只有  把   问题  想   得   一些      才能  疫情      传播

              is                                         
  ____________|_____________________________              
 |           way                            |            
 |    ________|___                          |             
 |   |   |       beat                       |            
 |   |   |     ___|_____                    |             
 |   |   |    |       spread                |            
 |   |   |    |    _____|_______            |             
 |   |   |    |   |             of        think          
 |   |   |    |   |             |       ____|_______      
 |   |   |   